In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import seaborn as sns
import requests

import wikipedia

In [57]:
df_all_input = pd.read_csv('Datasets/Original_ET50k_entity_vectors.csv')
df_all_input = df_all_input.set_index('Unnamed: 0')

In [16]:
df_et100_label = pd.read_csv('Datasets/ET100_with_Labels.csv')

# Wikidata Info Collection

In [49]:
import requests

def get_wikidata_id(wikipedia_article_name, lang='en'):
    """Fetch the Wikidata ID for a given Wikipedia article name."""
    url = f'https://{lang}.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'format': 'json',
        'titles': wikipedia_article_name,
        'prop': 'pageprops',
    }
    response = requests.get(url, params=params).json()
    page = next(iter(response['query']['pages'].values()))
    return page.get('pageprops', {}).get('wikibase_item')

def get_wikidata_info(wikidata_id):
    """Fetch Wikidata information using the Wikidata ID."""
    url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbgetentities',
        'ids': wikidata_id,
        'format': 'json',
        'languages': 'en',  # Adjust if you need information in another language
        'props': 'labels|descriptions|claims',  # Adjust based on the needed information
    }
    response = requests.get(url, params=params).json()
    return response['entities'][wikidata_id]



## Wikidata ID

In [ ]:
article_id_dict = {}
index = 0

In [ ]:
%%time
while index < len(df_all_input.index.tolist()):
    print (index)
    try:    
        article = df_all_input.index.tolist()[index]
        wikidata_id = get_wikidata_id(article)
        article_id_dict[article] = wikidata_id
        index = index + 1
    except KeyError as ke:
        print ('KeyError')
        print (article)
        index = index + 1

In [ ]:
df_wk_id = pd.DataFrame.from_dict(article_id_dict, orient = 'index')
df_wk_id = df_wk_id.reset_index()
df_wk_id.columns = ['Technology','Wikidata_ID']


In [52]:
# Remove Duplicates with #
df_wk_id_unique = df_wk_id.copy()
df_wk_id_unique['Technology'] = df_wk_id_unique['Technology'].apply(lambda x: x.split('#')[0].strip().replace('\xa0',' ').replace(' (mechanical)|machine','').replace('|Radiative recombination',''))
df_wk_id_unique = df_wk_id_unique.drop_duplicates()
print (df_wk_id_unique.shape)

# Remove Files
df_wk_id_unique = df_wk_id_unique[~df_wk_id_unique['Technology'].str.contains('File:')]
print (df_wk_id_unique.shape)
# Remove Images
df_wk_id_unique = df_wk_id_unique[~df_wk_id_unique['Technology'].str.contains('Image:')]
print (df_wk_id_unique.shape)
# Remove Portals
df_wk_id_unique = df_wk_id_unique[~df_wk_id_unique['Technology'].str.contains('Portal:')]
print (df_wk_id_unique.shape)
# Remove authors
df_wk_id_unique = df_wk_id_unique[~df_wk_id_unique['Technology'].str.contains('(author)')]
print (df_wk_id_unique.shape)
# Remove video game
df_wk_id_unique = df_wk_id_unique[~df_wk_id_unique['Technology'].str.contains('video game')]
print (df_wk_id_unique.shape)
# Remove ":" without Wikidata ID
df_wk_id_unique = df_wk_id_unique[~((df_wk_id_unique['Technology'].str.contains(':')) & (df_wk_id_unique['Wikidata_ID'].isna()))]
print (df_wk_id_unique.shape)

,Technology,Wikidata_ID
0,List of emerging technologies,Q2667876
1,Machine guidance,Q6723715
2,Gustatory technology,Q17054731
3,Differential technological development,Q5275369
4,Robotic paradigm,Q7353423
...,...,...
54687,Financial technology,NaN
54688,Genetics,Q7162
54689,Microsatellite,Q265193
54690,Online service provider,Q1641122


In [62]:
df_wk_id_unique_copy = df_wk_id_unique.copy()
missing_tech_list = df_wk_id_unique_copy[df_wk_id_unique_copy['Wikidata_ID'].isna()]['Technology'].tolist()

tech_index = 0
missing_id_dict = {}


In [72]:
while tech_index < len(missing_tech_list):
    print (tech_index)
    tech = missing_tech_list[tech_index]
    try:
        new_tech = wikipedia.search(tech)[0]
        wikidata_id = get_wikidata_id(new_tech)
        missing_id_dict[tech] = wikidata_id
        tech_index = tech_index + 1
    except IndexError:
        tech_index = tech_index + 1
    except wikipedia.exceptions.WikipediaException:
        tech_index = tech_index + 1

1555
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754


3194
3195
3196
3197
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3208
3209
3210
3211
3212
3213
3214
3215
3216
3217
3218
3219
3220
3221
3222
3223
3224
3225
3226
3227
3228
3229
3230
3231
3232
3233
3234
3235
3236
3237
3238
3239
3240
3241
3242
3243
3244
3245
3246
3247
3248
3249
3250
3251
3252
3253
3254
3255
3256
3257
3258
3259
3260
3261
3262
3263
3264
3265
3266
3267
3268
3269
3270
3271
3272
3273
3274
3275
3276
3277
3278
3279
3280
3281
3282
3283
3284
3285
3286
3287
3288
3289
3290
3291
3292
3293
3294
3295
3296
3297
3298
3299
3300
3301
3302
3303
3304
3305
3306
3307
3308
3309
3310
3311
3312
3313
3314
3315
3316
3317
3318
3319
3320
3321
3322
3323
3324
3325
3326
3327
3328
3329
3330
3331
3332
3333
3334
3335
3336
3337
3338
3339
3340
3341
3342
3343
3344
3345
3346
3347
3348
3349
3350
3351
3352
3353
3354
3355
3356
3357
3358
3359
3360
3361
3362
3363
3364
3365
3366
3367
3368
3369
3370
3371
3372
3373
3374
3375
3376
3377
3378
3379
3380
3381
3382
3383
3384
3385
3386
3387
3388
3389
3390
3391
3392
3393


In [73]:
df_wk_missing_id = pd.DataFrame.from_dict(missing_id_dict, orient = 'index')
df_wk_missing_id = df_wk_missing_id.reset_index()
df_wk_missing_id.columns = ['Technology','Wikidata_ID']
df_wk_missing_id.loc[df_wk_missing_id['Technology']=='Switching circuit theory','Wikidata_ID'] = 'Q7659118'
df_wk_missing_id = df_wk_missing_id[~df_wk_missing_id['Wikidata_ID'].isna()]

In [78]:
df_wk_missing_id.shape

(4032, 2)

In [88]:
df_wk_id_unique[df_wk_id_unique['Wikidata_ID'].isna()].shape[0]

4043

In [87]:
df_wk_id_unique.shape

(51818, 2)

In [103]:
df_wk_id_all = pd.concat([df_wk_id_unique[~df_wk_id_unique['Wikidata_ID'].isna()],df_wk_missing_id])
print (df_wk_id_all.shape)
df_wk_id_all = df_wk_id_all.drop_duplicates()
print (df_wk_id_all.shape)


(51806, 2)
(51805, 2)


In [181]:
df_wk_id_all.to_csv('Datasets/Original_ET50k_unique_Wikidata_ID.csv', index = False)

In [190]:
df_wk_id_all.shape

(51805, 2)

## Wikidata Info

In [119]:
wikidata_id_list = df_wk_id_all[~df_wk_id_all['Wikidata_ID'].isna()]['Wikidata_ID'].tolist()

In [120]:
len(wikidata_id_list)

51805

In [121]:
len(list(set(wikidata_id_list)))

51357

In [ ]:
id_index = 0
row_list = []
claims_index = {"instance of":'P31', "subclass of":'P279', "Commons category":'P373', 
                "topic's main category":'P910', "Freebase ID":'P646', "Microsoft Academic ID":'P6366',
                "OpenAlex ID":'P10283'}

In [ ]:
while id_index < len(wikidata_id_list):
    print (id_index)
    wikidata_id = wikidata_id_list[id_index]
    print (wikidata_id)
    wikidata_info = get_wikidata_info(wikidata_id)
    row_dict = {}
    row_dict['Wikidata_ID'] = wikidata_id
    if len(wikidata_info['descriptions']) != 0:
        row_dict['Description'] = wikidata_info['descriptions']['en']['value']
    else:
        row_dict['Description'] = np.nan
    for key, value in claims_index.items():
        if (value == 'P373') or (value == 'P646') or (value == 'P6366') or (value == 'P10283'):
            if value in list(wikidata_info['claims'].keys()):
                try:
                    row_dict[key] = ','.join([i['mainsnak']['datavalue']['value'] for i in wikidata_info['claims'][value]])
                except KeyError as ke:
                    row_dict[key] = np.nan
            else:
                row_dict[key] = np.nan
        else:
            if value in list(wikidata_info['claims'].keys()):
                try:
                    row_dict[key] = ','.join([i['mainsnak']['datavalue']['value']['id'] for i in wikidata_info['claims'][value]])
                except KeyError as ke:
                    row_dict[key] = np.nan
            else:
                row_dict[key] = np.nan
    row_list.append(row_dict)
    id_index = id_index + 1

In [ ]:
df_wikidata_info = pd.DataFrame(row_list)
df_wikidata_info.to_csv('Datasets/ET50k_Wikidata_info.csv', index = False)

## Wikidata Item Name

In [ ]:
instance_of_all_list = df_wikidata_info[~df_wikidata_info['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()

instance_of_list = [y for x in instance_of_all_list for y in x]

instance_unique_list = pd.Series(instance_of_list).value_counts().index.tolist()

print (len(instance_unique_list))

instance_index = 0
instance_of_row_list = []


In [ ]:
while instance_index < len(instance_unique_list):
    print (instance_index)
    wikidata_id = instance_unique_list[instance_index]
    row_dict = {}
    row_dict['Wikidata_ID'] = wikidata_id
    wikidata_info = get_wikidata_info(wikidata_id)
    try:
        row_dict['instance_of_label'] = wikidata_info['labels']['en']['value']
    except KeyError:
        row_dict['instance_of_label'] = np.nan
    instance_of_row_list.append(row_dict)
    instance_index = instance_index + 1

In [ ]:
df_instance_of_label = pd.DataFrame(instance_of_row_list)
df_instance_of_label.to_csv('Datasets/ET50k_Wikidata_instance_of_label.csv', index = False)

In [ ]:
subclass_of_all_list = df_wikidata_info[~df_wikidata_info['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()

subclass_of_list = [y for x in subclass_of_all_list for y in x]

subclass_unique_list = pd.Series(subclass_of_list).value_counts().index.tolist()

print (len(subclass_unique_list))

subclass_index = 0
subclass_of_row_list = []

In [ ]:
while subclass_index < len(subclass_unique_list):
    print (subclass_index)
    wikidata_id = subclass_unique_list[subclass_index]
    row_dict = {}
    row_dict['Wikidata_ID'] = wikidata_id
    wikidata_info = get_wikidata_info(wikidata_id)
    try:
        row_dict['subclass_of_label'] = wikidata_info['labels']['en']['value']
    except KeyError:
        row_dict['subclass_of_label'] = np.nan
    subclass_of_row_list.append(row_dict)
    subclass_index = subclass_index + 1

In [ ]:
df_subclass_of_label = pd.DataFrame(subclass_of_row_list)

In [ ]:
df_subclass_of_label.to_csv('Datasets/ET50k_Wikidata_subclass_of_label.csv', index = False)

## Wikidata Combined

In [174]:
df_wikidata_info = pd.read_csv('Datasets/ET50k_Wikidata_info.csv')

In [183]:
df_et_wk_info = df_wk_id_all.merge(df_wikidata_info, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')
df_et_wk_info_label = df_et_wk_info.copy()

instance_of_all_list = df_wikidata_info[~df_wikidata_info['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()
instance_of_list = [y for x in instance_of_all_list for y in x]
df_instance_counts = pd.DataFrame(pd.Series(instance_of_list).value_counts()).reset_index()
df_instance_counts.columns = ['Wikidata_ID','Counts']
df_instance_counts = df_instance_counts.merge(df_instance_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

subclass_of_all_list = df_wikidata_info[~df_wikidata_info['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()
subclass_of_list = [y for x in subclass_of_all_list for y in x]
df_subclass_counts = pd.DataFrame(pd.Series(subclass_of_list).value_counts()).reset_index()
df_subclass_counts.columns = ['Wikidata_ID','Counts']
df_subclass_counts = df_subclass_counts.merge(df_subclass_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')


instance_subs = df_instance_counts.set_index('Wikidata_ID')[['instance_of_label']].to_dict()['instance_of_label']
df_et_wk_info_label.loc[~df_et_wk_info_label['instance of'].isna(),'instance of'] = df_et_wk_info_label.loc[~df_et_wk_info_label['instance of'].isna(),'instance of'].apply(lambda x: ','.join([str(instance_subs.get(item,item)) for item in str(x).split(',')]))

subclass_subs = df_subclass_counts.set_index('Wikidata_ID')[['subclass_of_label']].to_dict()['subclass_of_label']
df_et_wk_info_label.loc[~df_et_wk_info_label['subclass of'].isna(),'subclass of'] = df_et_wk_info_label.loc[~df_et_wk_info_label['subclass of'].isna(),'subclass of'].apply(lambda x: ','.join([str(subclass_subs.get(item,item)) for item in str(x).split(',')]))

df_et_wk_info_label.to_csv('Datasets/ET50k_Wikidata_info_with_label.csv', index = False)

In [186]:
df_et_wk_info_label.shape

(51805, 10)

# Data Filtering

In [198]:
df_et100_label

,Technology,HC_4,HC_10
0,Data integration,Digital Business,Digital Transformation
1,Cloud computing,Digital Business,Digital Transformation
2,Marketing automation,Digital Business,Digital Transformation
3,Legal technology,Digital Business,Digital Transformation
4,Financial technology,Digital Business,Digital Transformation
...,...,...,...
95,Biotechnology,Healthy Living,Biomedicine and Biomaterials
96,Nanotechnology,Healthy Living,Biomedicine and Biomaterials
97,Biomaterial,Healthy Living,Biomedicine and Biomaterials
98,Synthetic biology,Healthy Living,Biomedicine and Biomaterials


In [ ]:
df_wk_id_all = pd.read_csv('Datasets/Original_ET50k_unique_Wikidata_ID.csv')
df_wikidata_info = pd.read_csv('Datasets/ET50k_Wikidata_info.csv')
df_instance_of_label = pd.read_csv('Datasets/ET50k_Wikidata_instance_of_label.csv')
df_subclass_of_label = pd.read_csv('Datasets/ET50k_Wikidata_subclass_of_label.csv')

df_et_wk_info = df_wk_id_all.merge(df_wikidata_info, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

remove_instance_list = ['human','business','organization','idiom','culture','state','ancient','style',
                        'historical','video game','public company','company','type of chemical entity','entity',
                        'research institute','institute','enterprise','written work','nonprofit organization',
                        'programming language','university','website','government agency','government','agency',
                        'school','standard','award','institution','taxon','history','file',
                        'format','film','conference','aspect in a geographic region','publisher',
                        'library','outline','laboratory','tabletop role-playing game','center','episode',
                        'phenomenon','magazine','brand','project','television','instrument','industry','fictional',
                        'faculty','language','quality','experiment','society','book',
                        'activity','paradox','consortium','law','hospital','timeline','comparison','equation',
                        'park','museum','subsidiary','ship','disease','database','association','bus','event',
                        'reaction','album','college','formula','polymer','compiler','division','station','venture',
                        'canal','strategy','occupation','lecture','rule','road','service','chair','newspaper',
                        'collection','task','policy','diagram','galaxy','data set','prefix','island','publication',
                        'function','news website','literary work','scientific journal','country','zone']
remove_instance_list = [i.lower() for i in remove_instance_list]


remove_subclass_list = ['smartphone','mobile phone','formula one car','diagram','philosophy','software testing',
                        'chemical compound','ethics','programming language','strategy','error',
                        'ieee 802','fallacy','goods','iso standard','grammar','memory','as a service',
                        'definition','heater','toy']
remove_subclass_list = [i.lower() for i in remove_subclass_list]

# Remove instance


In [243]:
all_instance_of = df_et_wk_info[~df_et_wk_info['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()
instance_of_list = [y for x in all_instance_of for y in x]
df_instance_counts = pd.DataFrame(pd.Series(instance_of_list).value_counts()).reset_index()
df_instance_counts.columns = ['Wikidata_ID','Counts']
df_instance_counts = df_instance_counts.merge(df_instance_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

all_subclass_of = df_et_wk_info[~df_et_wk_info['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()
subclass_of_list = [y for x in all_subclass_of for y in x]
df_subclass_counts = pd.DataFrame(pd.Series(subclass_of_list).value_counts()).reset_index()
df_subclass_counts.columns = ['Wikidata_ID','Counts']
df_subclass_counts = df_subclass_counts.merge(df_subclass_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')


In [254]:
df_instance_counts.to_csv('Datasets/ET50k_Wikidata_instance_of_label_counts.csv', index = False)
df_subclass_counts.to_csv('Datasets/ET50k_Wikidata_subclass_of_label_counts.csv', index = False)

In [258]:
df_keep_tech = df_et_wk_info[df_et_wk_info['Technology'].isin(df_et100_label['Technology'])]
keep_instance_of = df_keep_tech[~df_keep_tech['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()
keep_instance_of_list = [y for x in keep_instance_of for y in x]
df_keep_instance_counts = pd.DataFrame(pd.Series(keep_instance_of_list).value_counts()).reset_index()
df_keep_instance_counts.columns = ['Wikidata_ID','Counts']
df_keep_instance_counts = df_keep_instance_counts.merge(df_instance_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

keep_subclass_of = df_keep_tech[~df_keep_tech['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()
keep_subclass_of_list = [y for x in keep_subclass_of for y in x]
df_keep_subclass_counts = pd.DataFrame(pd.Series(keep_subclass_of_list).value_counts()).reset_index()
df_keep_subclass_counts.columns = ['Wikidata_ID','Counts']
df_keep_subclass_counts = df_keep_subclass_counts.merge(df_subclass_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

df_keep_instance_counts.to_csv('Datasets/ET100_Wikidata_instance_of_label_counts.csv', index = False)
df_keep_subclass_counts.to_csv('Datasets/ET100_Wikidata_subclass_of_label_counts.csv', index = False)

In [256]:
df_keep_instance_counts.shape

(49, 3)

In [257]:
df_keep_subclass_counts.shape

(124, 3)

# External Technology List

In [248]:
df_potential_tech1 = pd.read_excel('Datasets/Lists of Emerging Technologies and other Technologies.xlsx', sheet_name = '1. ET List (Complete N=242)')
df_potential_tech2 = pd.read_excel('Datasets/Lists of Emerging Technologies and other Technologies.xlsx', sheet_name = '2. WP Articles with Q11016 (tec')
df_potential_tech3 = pd.read_excel('Datasets/Lists of Emerging Technologies and other Technologies.xlsx', sheet_name = '3. XT List (Selected Types N=3,')


In [249]:
df_potential_tech1

,Emerging Tech List (n=242)
0,https://en.wikipedia.org/wiki/3D_optical_data_...
1,https://en.wikipedia.org/wiki/4D_printing
2,https://en.wikipedia.org/wiki/6G_(network)
3,https://en.wikipedia.org/wiki/Active_structure
4,https://en.wikipedia.org/wiki/Adaptive_Complia...
...,...
237,https://en.wikipedia.org/wiki/Vortex_engine
238,https://en.wikipedia.org/wiki/Wetware_computer
239,https://en.wikipedia.org/wiki/Wireless_energy_...
240,https://en.wikipedia.org/wiki/X-53_Active_Aero...


In [250]:
df_potential_tech2

,item,itemLabel,itemDescription,article
0,http://www.wikidata.org/entity/Q83787,Dubai Metro,"Rapid Transit Train in Dubai, United Arab Emir...",https://en.wikipedia.org/wiki/Dubai_Metro
1,http://www.wikidata.org/entity/Q94887,Khan Academy,non-profit educational organization,https://en.wikipedia.org/wiki/Khan_Academy
2,http://www.wikidata.org/entity/Q104534,digital subscriber line,technologies for transmission of digital data ...,https://en.wikipedia.org/wiki/Digital_subscrib...
3,http://www.wikidata.org/entity/Q111586,LowFER,experimental radio communication,https://en.wikipedia.org/wiki/LowFER
4,http://www.wikidata.org/entity/Q114162,Intermediate film system,NaN,https://en.wikipedia.org/wiki/Intermediate_fil...
...,...,...,...,...
2496,http://www.wikidata.org/entity/Q125252763,Nheengatu App,Brazilian online platform for Nheengatu learning,NaN
2497,http://www.wikidata.org/entity/Q125360381,Basaltsteinbruch Am HohenbÃ¼hl,large-scale natural monument of a former quarr...,NaN
2498,http://www.wikidata.org/entity/Q125361979,Feuchtbiotop und Steinbruch Der Ziegenberg,extensive natural monument of a moist soil bas...,NaN
2499,http://www.wikidata.org/entity/Q125363240,Leuner Burg,natural monument of a former quarry with an op...,NaN


In [251]:
df_potential_tech3

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,WP Article,Subcategory (WP)
0,1.0,Agricultural_engineering,4833.0,230 / day,https://en.wikipedia.org/wiki/Agricultural_eng...,Agricultural Technologies
1,2.0,Precision_agriculture,3945.0,188 / day,https://en.wikipedia.org/wiki/Precision_agricu...,Agricultural Technologies
2,3.0,Incubator_(egg),2366.0,113 / day,https://en.wikipedia.org/wiki/Incubator_(egg),Agricultural Technologies
3,4.0,Agricultural_technology,2046.0,97 / day,https://en.wikipedia.org/wiki/Agricultural_tec...,Agricultural Technologies
4,5.0,Digital_agriculture,1852.0,88 / day,https://en.wikipedia.org/wiki/Digital_agriculture,Agricultural Technologies
...,...,...,...,...,...,...
3289,104.0,In_situ_thermal_desorption,19.0,1 / day,https://en.wikipedia.org/wiki/In_situ_thermal_...,Waste treatment technology
3290,105.0,New_Technologies_Demonstrator_Programme,14.0,1 / day,https://en.wikipedia.org/wiki/New_Technologies...,Waste treatment technology
3291,106.0,Fluorescent_lamp_crusher,13.0,1 / day,https://en.wikipedia.org/wiki/Fluorescent_lamp...,Waste treatment technology
3292,107.0,Scottish_Landfill_Tax,12.0,1 / day,https://en.wikipedia.org/wiki/Scottish_Landfil...,Waste treatment technology


In [247]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter("Check_instance_subclass_Filter.xlsx", engine="xlsxwriter")

# Write each dataframe to a different worksheet.
df_instance_counts.to_excel(writer, sheet_name="instance of", index = False)
df_subclass_counts.to_excel(writer, sheet_name="subclass of", index = False)

# Close the Pandas Excel writer and output the Excel file.
writer.close()

In [236]:
# for i in df_instance_counts[(~df_instance_counts['Wikidata_ID'].isin(df_keep_instance_counts['Wikidata_ID'])) & (df_instance_counts['instance_of_label'].str.contains('technolo'))]['instance_of_label'].tolist():
#     print (i)


In [228]:
# for i in df_instance_counts[~df_instance_counts['Wikidata_ID'].isin(df_keep_instance_counts['Wikidata_ID'])]['instance_of_label'].tolist():
#     print (i)

In [ ]:
all_instance_of_part1 = df_part1[~df_part1['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()
instance_of_part1_list = [y for x in all_instance_of_part1 for y in x]
df_instance_part1_counts = pd.DataFrame(pd.Series(instance_of_part1_list).value_counts()).reset_index()
df_instance_part1_counts.columns = ['Wikidata_ID','Counts']
df_instance_part1_counts = df_instance_part1_counts.merge(df_instance_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

df_instance_part1_counts = df_instance_part1_counts[~df_instance_part1_counts['instance_of_label'].isna()]
df_instance_part1_counts['instance_of_label'] = df_instance_part1_counts['instance_of_label'].apply(lambda x: str(x).lower())

for i in remove_instance_list:
    df_instance_part1_counts = df_instance_part1_counts[~df_instance_part1_counts['instance_of_label'].str.contains(i)]
    
df_part1 = df_part1.loc[(df_part1['instance of'].isna()) | (df_part1['instance of'].str.contains('|'.join(df_instance_part1_counts['Wikidata_ID'].tolist()))),:]
print (df_part1.shape)


In [ ]:
df_wk_id_unique_copy = df_wk_id_unique.copy()
df_wikidata_info = pd.read_csv('Datasets/ET50k_Wikidata_info.csv')
df_instance_of_label = pd.read_csv('Datasets/ET50k_Wikidata_instance_of_label.csv')
df_subclass_of_label = pd.read_csv('Datasets/ET50k_Wikidata_subclass_of_label.csv')

df_et_wk_info = df_wk_id_unique_copy.merge(df_wikidata_info, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

remove_instance_list = ['human','business','organization','idiom','culture','state','ancient','style',
                        'historical','video game','public company','company','type of chemical entity','entity',
                        'research institute','institute','enterprise','written work','nonprofit organization',
                        'programming language','university','website','government agency','government','agency',
                        'school','standard','award','institution','taxon','history','file',
                        'format','film','conference','aspect in a geographic region','publisher',
                        'library','outline','laboratory','tabletop role-playing game','center','episode',
                        'phenomenon','magazine','brand','project','television','instrument','industry','fictional',
                        'faculty','language','quality','experiment','society','book',
                        'activity','paradox','consortium','law','hospital','timeline','comparison','equation',
                        'park','museum','subsidiary','ship','disease','database','association','bus','event',
                        'reaction','album','college','formula','polymer','compiler','division','station','venture',
                        'canal','strategy','occupation','lecture','rule','road','service','chair','newspaper',
                        'collection','task','policy','diagram','galaxy','data set','prefix','island','publication',
                        'function','news website','literary work','scientific journal','country','zone']
remove_instance_list = [i.lower() for i in remove_instance_list]


remove_subclass_list = ['smartphone','mobile phone','formula one car','diagram','philosophy','software testing',
                        'chemical compound','ethics','programming language','strategy','error',
                        'ieee 802','fallacy','goods','iso standard','grammar','memory','as a service',
                        'definition','heater','toy']
remove_subclass_list = [i.lower() for i in remove_subclass_list]

df_descr_tech = df_et_wk_info[(~df_et_wk_info['Description'].isna()) & (df_et_wk_info['Description'].str.contains('tech'))]
df_descr_no_tech = df_et_wk_info.iloc[~df_et_wk_info.index.isin(df_descr_tech.index.tolist())]

# Part1
df_part1 = df_descr_tech.copy()
print (df_part1.shape)

# Remove instance
all_instance_of_part1 = df_part1[~df_part1['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()
instance_of_part1_list = [y for x in all_instance_of_part1 for y in x]
df_instance_part1_counts = pd.DataFrame(pd.Series(instance_of_part1_list).value_counts()).reset_index()
df_instance_part1_counts.columns = ['Wikidata_ID','Counts']
df_instance_part1_counts = df_instance_part1_counts.merge(df_instance_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

df_instance_part1_counts = df_instance_part1_counts[~df_instance_part1_counts['instance_of_label'].isna()]
df_instance_part1_counts['instance_of_label'] = df_instance_part1_counts['instance_of_label'].apply(lambda x: str(x).lower())

for i in remove_instance_list:
    df_instance_part1_counts = df_instance_part1_counts[~df_instance_part1_counts['instance_of_label'].str.contains(i)]
    
df_part1 = df_part1.loc[(df_part1['instance of'].isna()) | (df_part1['instance of'].str.contains('|'.join(df_instance_part1_counts['Wikidata_ID'].tolist()))),:]
print (df_part1.shape)

# Remove subclass
all_subclass_of_part1 = df_part1[~df_part1['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()
subclass_of_part1_list = [y for x in all_subclass_of_part1 for y in x]
df_subclass_part1_counts = pd.DataFrame(pd.Series(subclass_of_part1_list).value_counts()).reset_index()
df_subclass_part1_counts.columns = ['Wikidata_ID','Counts']
df_subclass_part1_counts = df_subclass_part1_counts.merge(df_subclass_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

df_subclass_part1_counts = df_subclass_part1_counts[~df_subclass_part1_counts['subclass_of_label'].isna()]
df_subclass_part1_counts['subclass_of_label'] = df_subclass_part1_counts['subclass_of_label'].apply(lambda x: str(x).lower())
for i in remove_subclass_list:
    df_subclass_part1_counts = df_subclass_part1_counts[~df_subclass_part1_counts['subclass_of_label'].str.contains(i)]
    
df_part1 = df_part1.loc[(df_part1['subclass of'].isna()) | (df_part1['subclass of'].str.contains('|'.join(df_subclass_part1_counts['Wikidata_ID'].tolist()))),:]
print (df_part1.shape)

# Part2
df_part2 = df_descr_no_tech.copy()
print (df_part2.shape)

# Remove instance
all_instance_of_part2 = df_part2[~df_part2['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()
instance_of_part2_list = [y for x in all_instance_of_part2 for y in x]
df_instance_part2_counts = pd.DataFrame(pd.Series(instance_of_part2_list).value_counts()).reset_index()
df_instance_part2_counts.columns = ['Wikidata_ID','Counts']
df_instance_part2_counts = df_instance_part2_counts.merge(df_instance_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

df_instance_part2_counts = df_instance_part2_counts[~df_instance_part2_counts['instance_of_label'].isna()]
df_instance_part2_counts['instance_of_label'] = df_instance_part2_counts['instance_of_label'].apply(lambda x: str(x).lower())

for i in remove_instance_list:
    df_instance_part2_counts = df_instance_part2_counts[~df_instance_part2_counts['instance_of_label'].str.contains(i)]
    
df_part2 = df_part2.loc[(df_part2['instance of'].isna()) | (df_part2['instance of'].str.contains('|'.join(df_instance_part2_counts['Wikidata_ID'].tolist()))),:]
print (df_part2.shape)

# Remove subclass
all_subclass_of_part2 = df_part2[~df_part2['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()
subclass_of_part2_list = [y for x in all_subclass_of_part2 for y in x]
df_subclass_part2_counts = pd.DataFrame(pd.Series(subclass_of_part2_list).value_counts()).reset_index()
df_subclass_part2_counts.columns = ['Wikidata_ID','Counts']
df_subclass_part2_counts = df_subclass_part2_counts.merge(df_subclass_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

df_subclass_part2_counts = df_subclass_part2_counts[~df_subclass_part2_counts['subclass_of_label'].isna()]
df_subclass_part2_counts['subclass_of_label'] = df_subclass_part2_counts['subclass_of_label'].apply(lambda x: str(x).lower())
for i in remove_subclass_list:
    df_subclass_part2_counts = df_subclass_part2_counts[~df_subclass_part2_counts['subclass_of_label'].str.contains(i)]
    
df_part2 = df_part2.loc[(df_part2['subclass of'].isna()) | (df_part2['subclass of'].str.contains('|'.join(df_subclass_part2_counts['Wikidata_ID'].tolist()))),:]
print (df_part2.shape)

df_wikidata_filter = pd.concat([df_part1, df_part2])
print (df_wikidata_filter.shape)

## Missing Part

In [ ]:
df_wk_missing_id

In [104]:
wikidata_id_list = df_wk_missing_id[~df_wk_missing_id['Wikidata_ID'].isna()]['Wikidata_ID'].tolist()

In [105]:
len(wikidata_id_list)

4031

In [106]:
id_index = 0
row_list = []
claims_index = {"instance of":'P31', "subclass of":'P279', "Commons category":'P373', 
                "topic's main category":'P910', "Freebase ID":'P646', "Microsoft Academic ID":'P6366',
                "OpenAlex ID":'P10283'}

In [107]:
while id_index < len(wikidata_id_list):
    print (id_index)
    wikidata_id = wikidata_id_list[id_index]
    print (wikidata_id)
    wikidata_info = get_wikidata_info(wikidata_id)
    row_dict = {}
    row_dict['Wikidata_ID'] = wikidata_id
    if len(wikidata_info['descriptions']) != 0:
        row_dict['Description'] = wikidata_info['descriptions']['en']['value']
    else:
        row_dict['Description'] = np.nan
    for key, value in claims_index.items():
        if (value == 'P373') or (value == 'P646') or (value == 'P6366') or (value == 'P10283'):
            if value in list(wikidata_info['claims'].keys()):
                try:
                    row_dict[key] = ','.join([i['mainsnak']['datavalue']['value'] for i in wikidata_info['claims'][value]])
                except KeyError as ke:
                    row_dict[key] = np.nan
            else:
                row_dict[key] = np.nan
        else:
            if value in list(wikidata_info['claims'].keys()):
                try:
                    row_dict[key] = ','.join([i['mainsnak']['datavalue']['value']['id'] for i in wikidata_info['claims'][value]])
                except KeyError as ke:
                    row_dict[key] = np.nan
            else:
                row_dict[key] = np.nan
    row_list.append(row_dict)
    id_index = id_index + 1

0
Q17097955
1
Q21715262
2
Q4833645
3
Q4182287
4
Q40890078
5
Q7578910
6
Q18394818
7
Q5532384
8
Q22192
9
Q953727
10
Q6808941
11
Q1078276
12
Q30598483
13
Q4643915
14
Q92855
15
Q503520
16
Q610398
17
Q1540232
18
Q2735691
19
Q39619
20
Q18276277
21
Q7096419
22
Q192776
23
Q4378395
24
Q104880358
25
Q18378458
26
Q17625380
27
Q3985153
28
Q2577671
29
Q3312428
30
Q4801079
31
Q324963
32
Q5270587
33
Q1200665
34
Q176890
35
Q4652026
36
Q2294986
37
Q6954925
38
Q5227304
39
Q5547558
40
Q7302657
41
Q2303838
42
Q2111831
43
Q25104146
44
Q1076157
45
Q60740481
46
Q1434934
47
Q1895360
48
Q17105190
49
Q4801081
50
Q3428507
51
Q6804076
52
Q24883926
53
Q7662740
54
Q2144042
55
Q98442705
56
Q3296256
57
Q2025765
58
Q1095907
59
Q5275034
60
Q176197
61
Q2293381
62
Q2577671
63
Q5871967
64
Q4883044
65
Q22192
66
Q1333980
67
Q21190892
68
Q921015
69
Q837718
70
Q25169
71
Q221681
72
Q17149283
73
Q128896
74
Q15855659
75
Q21068221
76
Q1405050
77
Q711868
78
Q1122789
79
Q1952355
80
Q124420612
81
Q4437154
82
Q537952
83
Q23038294
84


645
Q24965573
646
Q2652244
647
Q690689
648
Q553799
649
Q3500657
650
Q1182555
651
Q16663756
652
Q208443
653
Q4121257
654
Q243606
655
Q7876097
656
Q131012
657
Q904576
658
Q28169333
659
Q248998
660
Q1673713
661
Q1185128
662
Q681875
663
Q7553552
664
Q815846
665
Q79037
666
Q1333561
667
Q502067
668
Q3220391
669
Q412563
670
Q3152227
671
Q5264349
672
Q4781539
673
Q5227135
674
Q869122
675
Q2486226
676
Q208376
677
Q3297086
678
Q7711822
679
Q5275374
680
Q4290703
681
Q1396395
682
Q11255
683
Q19820
684
Q3101701
685
Q898312
686
Q5968893
687
Q16395217
688
Q305932
689
Q7140437
690
Q478019
691
Q6713673
692
Q16982455
693
Q7532148
694
Q4037807
695
Q23580638
696
Q362601
697
Q2743938
698
Q176789
699
Q38609561
700
Q232546
701
Q596193
702
Q5130498
703
Q1757054
704
Q274586
705
Q15984000
706
Q106041063
707
Q3495015
708
Q354217
709
Q4866219
710
Q3520778
711
Q17105319
712
Q1453575
713
Q7309746
714
Q18125864
715
Q466977
716
Q2283
717
Q25245349
718
Q2554325
719
Q384596
720
Q2069335
721
Q2992422
722
Q832152
723
Q51

1259
Q6887148
1260
Q24963577
1261
Q1928527
1262
Q7205005
1263
Q622779
1264
Q17055549
1265
Q28451729
1266
Q24949777
1267
Q841918
1268
Q7189550
1269
Q7544146
1270
Q5065899
1271
Q543275
1272
Q517117
1273
Q18207172
1274
Q1385229
1275
Q7243252
1276
Q837060
1277
Q17140727
1278
Q4733755
1279
Q6418879
1280
Q6028924
1281
Q1809949
1282
Q77057
1283
Q13597976
1284
Q235219
1285
Q8035855
1286
Q7817
1287
Q7853895
1288
Q2167279
1289
Q462154
1290
Q859398
1291
Q361431
1292
Q14623169
1293
Q18471030
1294
Q7705696
1295
Q830450
1296
Q15148967
1297
Q6626685
1298
Q1072552
1299
Q2356163
1300
Q2564903
1301
Q5012997
1302
Q3181728
1303
Q21015850
1304
Q10986919
1305
Q145874
1306
Q1362150
1307
Q1434934
1308
Q3093182
1309
Q4809995
1310
Q6894763
1311
Q5206024
1312
Q643635
1313
Q58455
1314
Q4950687
1315
Q17078059
1316
Q81571
1317
Q20795192
1318
Q5157427
1319
Q25324637
1320
Q8034614
1321
Q2975790
1322
Q18125952
1323
Q7460785
1324
Q7370302
1325
Q2750576
1326
Q24964334
1327
Q1146298
1328
Q144128
1329
Q5099768
1330
Q25245

1846
Q944633
1847
Q863823
1848
Q3304308
1849
Q2880130
1850
Q167532
1851
Q905870
1852
Q5602501
1853
Q25245533
1854
Q7554205
1855
Q24693821
1856
Q3566045
1857
Q2545179
1858
Q7440467
1859
Q641659
1860
Q1550565
1861
Q740034
1862
Q2211614
1863
Q6608059
1864
Q3346318
1865
Q379523
1866
Q5242423
1867
Q461992
1868
Q5421008
1869
Q25313068
1870
Q284636
1871
Q1665604
1872
Q28976355
1873
Q52112789
1874
Q111835586
1875
Q7585631
1876
Q2945440
1877
Q3239172
1878
Q5513417
1879
Q914874
1880
Q14631
1881
Q1151468
1882
Q4392761
1883
Q5710515
1884
Q5275005
1885
Q507234
1886
Q7097769
1887
Q7438846
1888
Q3648676
1889
Q1931479
1890
Q18388892
1891
Q18405127
1892
Q7000431
1893
Q5357840
1894
Q756435
1895
Q5165788
1896
Q4037082
1897
Q16664756
1898
Q5578510
1899
Q5013945
1900
Q17084036
1901
Q119307840
1902
Q12180017
1903
Q3595265
1904
Q26252665
1905
Q4045970
1906
Q2438127
1907
Q6030438
1908
Q7162911
1909
Q5689285
1910
Q5450261
1911
Q1019072
1912
Q203516
1913
Q80585
1914
Q5123055
1915
Q24770343
1916
Q701261
1917
Q54

2434
Q19512294
2435
Q389772
2436
Q1454771
2437
Q2566976
2438
Q16846719
2439
Q1143006
2440
Q21190504
2441
Q5132334
2442
Q4204279
2443
Q285927
2444
Q1772737
2445
Q840202
2446
Q5566764
2447
Q4811702
2448
Q4906677
2449
Q1282116
2450
Q24089757
2451
Q1279263
2452
Q241433
2453
Q6749265
2454
Q1853339
2455
Q830170
2456
Q461524
2457
Q2417914
2458
Q17512031
2459
Q2599574
2460
Q305918
2461
Q5308852
2462
Q797508
2463
Q16159499
2464
Q1254342
2465
Q7342120
2466
Q31207
2467
Q6634467
2468
Q2682133
2469
Q19561006
2470
Q298058
2471
Q2301555
2472
Q1132207
2473
Q623543
2474
Q22004920
2475
Q170323
2476
Q3813277
2477
Q856457
2478
Q484514
2479
Q15643802
2480
Q1524124
2481
Q102406672
2482
Q200101
2483
Q1516247
2484
Q5013369
2485
Q6643104
2486
Q905815
2487
Q7569143
2488
Q17152132
2489
Q3235927
2490
Q844764
2491
Q4785541
2492
Q3046028
2493
Q3965952
2494
Q6604905
2495
Q1076779
2496
Q6834793
2497
Q7171619
2498
Q20738516
2499
Q19483216
2500
Q3509385
2501
Q7860707
2502
Q4111450
2503
Q7612893
2504
Q17157156
2505
Q183

3022
Q1194317
3023
Q2271290
3024
Q17030799
3025
Q2988778
3026
Q17020273
3027
Q2273453
3028
Q17135794
3029
Q18619463
3030
Q862228
3031
Q4714638
3032
Q188639
3033
Q1675896
3034
Q8047675
3035
Q742866
3036
Q1778355
3037
Q118689
3038
Q1470887
3039
Q5305200
3040
Q1895288
3041
Q165400
3042
Q7039311
3043
Q1437507
3044
Q2273687
3045
Q5006165
3046
Q890875
3047
Q2021257
3048
Q14902502
3049
Q5953270
3050
Q7936148
3051
Q11974939
3052
Q3062459
3053
Q5988221
3054
Q7895367
3055
Q7975679
3056
Q6173703
3057
Q47490937
3058
Q55615812
3059
Q2166744
3060
Q6901575
3061
Q3816785
3062
Q1923162
3063
Q3628244
3064
Q409485
3065
Q1003792
3066
Q20922293
3067
Q5276054
3068
Q4090608
3069
Q6659496
3070
Q173801
3071
Q2475675
3072
Q478798
3073
Q7554963
3074
Q958460
3075
Q7619050
3076
Q702499
3077
Q42417289
3078
Q2900186
3079
Q65379
3080
Q6956112
3081
Q15044673
3082
Q487378
3083
Q23579690
3084
Q3555595
3085
Q17037714
3086
Q4202080
3087
Q1194650
3088
Q145490
3089
Q3306008
3090
Q22025522
3091
Q7653094
3092
Q182496
3093
Q50

3610
Q4159550
3611
Q922151
3612
Q122731
3613
Q7895298
3614
Q22909082
3615
Q4423908
3616
Q604924
3617
Q283780
3618
Q4129097
3619
Q663932
3620
Q16252033
3621
Q5235846
3622
Q3174091
3623
Q4818196
3624
Q5275880
3625
Q56275466
3626
Q16859272
3627
Q7665663
3628
Q5494243
3629
Q1787166
3630
Q19945990
3631
Q230945
3632
Q11307061
3633
Q860740
3634
Q17102557
3635
Q4537978
3636
Q1027984
3637
Q17006246
3638
Q12375202
3639
Q288156
3640
Q8023663
3641
Q582616
3642
Q224287
3643
Q6634477
3644
Q19868032
3645
Q1477251
3646
Q5155964
3647
Q5749018
3648
Q187854
3649
Q24456219
3650
Q3661247
3651
Q7189688
3652
Q6916733
3653
Q1341791
3654
Q55849612
3655
Q5162852
3656
Q3775506
3657
Q1750812
3658
Q9334
3659
Q15820293
3660
Q130652
3661
Q1207598
3662
Q12060721
3663
Q6565883
3664
Q1370704
3665
Q20204940
3666
Q1139749
3667
Q1942166
3668
Q542836
3669
Q5062514
3670
Q483915
3671
Q11181597
3672
Q846664
3673
Q681855
3674
Q7661132
3675
Q5732300
3676
Q694742
3677
Q4116214
3678
Q2231042
3679
Q124819461
3680
Q16207991
3681
Q2

In [108]:
df_wikidata_info = pd.DataFrame(row_list)
df_wikidata_info.to_csv('Datasets/ET50k_Wikidata_info_Missing_Part.csv', index = False)

In [110]:
df_wikidata_info_origin = pd.read_csv('Datasets/ET50k_Wikidata_info.csv')

In [125]:
df_wikidata_info_all = pd.concat([df_wikidata_info_origin, df_wikidata_info])
df_wikidata_info_all = df_wikidata_info_all.drop_duplicates()
df_wikidata_info_all.reset_index(drop = True, inplace = True)
df_wikidata_info_all = df_wikidata_info_all.drop([10088,18206,48431,47842,23080,32846])

In [144]:
df_wikidata_info_all.to_csv('Datasets/ET50k_Wikidata_info.csv', index = False)

In [152]:
df_instance_of_label_origin = pd.read_csv('Datasets/ET50k_Wikidata_instance_of_label.csv')

instance_of_all_list = df_wikidata_info_all[~df_wikidata_info_all['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()

instance_of_list = [y for x in instance_of_all_list for y in x]

instance_unique_list = pd.Series(instance_of_list).value_counts().index.tolist()

instance_unique_list = [i for i in instance_unique_list if i not in df_instance_of_label_origin['Wikidata_ID'].tolist()]

print (len(instance_unique_list))

instance_index = 0
instance_of_row_list = []


416


In [153]:
while instance_index < len(instance_unique_list):
    print (instance_index)
    wikidata_id = instance_unique_list[instance_index]
    row_dict = {}
    row_dict['Wikidata_ID'] = wikidata_id
    wikidata_info = get_wikidata_info(wikidata_id)
    try:
        row_dict['instance_of_label'] = wikidata_info['labels']['en']['value']
    except KeyError:
        row_dict['instance_of_label'] = np.nan
    instance_of_row_list.append(row_dict)
    instance_index = instance_index + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [154]:
df_instance_of_label = pd.DataFrame(instance_of_row_list)
# df_instance_of_label.to_csv('Datasets/ET50k_Wikidata_instance_of_label.csv', index = False)

In [155]:
df_instance_of_label_all = pd.concat([df_instance_of_label_origin, df_instance_of_label])

In [158]:
df_instance_of_label_all.shape

(5600, 2)

In [159]:
df_instance_of_label_all.to_csv('Datasets/ET50k_Wikidata_instance_of_label.csv', index = False)

In [160]:
df_subclass_of_label_origin = pd.read_csv('Datasets/ET50k_Wikidata_subclass_of_label.csv')

subclass_of_all_list = df_wikidata_info_all[~df_wikidata_info_all['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()

subclass_of_list = [y for x in subclass_of_all_list for y in x]

subclass_unique_list = pd.Series(subclass_of_list).value_counts().index.tolist()

subclass_unique_list = [i for i in subclass_unique_list if i not in df_subclass_of_label_origin['Wikidata_ID'].tolist()]

print (len(subclass_unique_list))

subclass_index = 0
subclass_of_row_list = []

462


In [161]:
while subclass_index < len(subclass_unique_list):
    print (subclass_index)
    wikidata_id = subclass_unique_list[subclass_index]
    row_dict = {}
    row_dict['Wikidata_ID'] = wikidata_id
    wikidata_info = get_wikidata_info(wikidata_id)
    try:
        row_dict['subclass_of_label'] = wikidata_info['labels']['en']['value']
    except KeyError:
        row_dict['subclass_of_label'] = np.nan
    subclass_of_row_list.append(row_dict)
    subclass_index = subclass_index + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [162]:
df_subclass_of_label = pd.DataFrame(subclass_of_row_list)
# df_subclass_of_label.to_csv('Datasets/ET50k_Wikidata_subclass_of_label.csv', index = False)

In [163]:
df_subclass_of_label_all = pd.concat([df_subclass_of_label_origin, df_subclass_of_label])

In [164]:
df_subclass_of_label_origin.shape

(8085, 2)

In [165]:
df_subclass_of_label_all.shape

(8547, 2)

In [167]:
df_subclass_of_label_all.to_csv('Datasets/ET50k_Wikidata_subclass_of_label.csv', index = False)

In [168]:
df_subclass_of_label_all.shape

(8547, 2)

In [142]:

df_wikidata_info_all['Wikidata_ID'].value_counts().shape

(51357,)

In [113]:
df_wikidata_info_origin.shape[0] + df_wikidata_info.shape[0]

51806

In [ ]:
df_wikidata_info_origin

In [118]:
df_wikidata_info[df_wikidata_info['Wikidata_ID'].isin(df_wikidata_info_origin['Wikidata_ID'])].shape

(387, 9)

In [117]:
df_wikidata_info_origin[df_wikidata_info_origin['Wikidata_ID'].isin(df_wikidata_info['Wikidata_ID'])]

,Wikidata_ID,Description,instance of,subclass of,Commons category,topic's main category,Freebase ID,Microsoft Academic ID,OpenAlex ID
99,Q197536,branch of machine learning,NaN,Q111862379,Deep learning,Q24070291,/m/0h1fn8h,108583219,"C108583219,C2984842247"
140,Q18276277,a person or entity that exists in a state beyo...,Q151885,NaN,Fictional or speculative posthumans,NaN,/m/03yc4_k,2779549840,C2779549840
143,Q176789,model for decision making under uncertainty,NaN,"Q1331926,Q176737",NaN,NaN,/m/048gl8,106189395,C106189395
326,Q189223,test of a machine's ability to exhibit intelli...,"Q96273743,Q24034552",Q1003030,Turing test,Q9691252,"/m/0b_42g,/m/0b_42",577917,C577917
360,Q2122216,interdisciplinary theory behind quantum computing,"Q4162444,Q1936384",Q21198,Quantum computing,Q7066518,/m/030gkt,5320026,"C5320026,C2987822364"
...,...,...,...,...,...,...,...,...,...
46920,Q30267,programming paradigm,"Q188267,Q66747126","Q2465506,Q80006",NaN,Q8271365,/m/012gcp,60051680,C60051680
47036,Q228502,selective restriction of access to a place or ...,"Q1799072,Q151885,Q5157582,Q96758092,Q1047113,Q...","Q1632136,Q95722120,Q119822504",Access control systems,Q8219526,/m/0b494,527821871,C527821871
47239,Q58734,introduction of contaminants into the natural ...,NaN,"Q2144359,Q1135789,Q275372,Q214609",Pollution,Q7144129,/m/066xq,521259446,"C521259446,C2909468537,C3017866148"
47344,Q3338539,1996 collectible card game designed by Richar...,Q734698,NaN,NaN,NaN,/m/01j7k1,NaN,NaN


In [5]:
df_wk_id_unique_copy = df_wk_id_unique.copy()
df_wikidata_info = pd.read_csv('Datasets/ET50k_Wikidata_info.csv')
df_instance_of_label = pd.read_csv('Datasets/ET50k_Wikidata_instance_of_label.csv')
df_subclass_of_label = pd.read_csv('Datasets/ET50k_Wikidata_subclass_of_label.csv')

df_et_wk_info = df_wk_id_unique_copy.merge(df_wikidata_info, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')



In [53]:
df_wikidata_info.shape

(47775, 9)

In [8]:
df_wk_id_unique_copy.shape

(51818, 2)

In [13]:
df_wk_id_unique_copy

,Technology,Wikidata_ID
0,List of emerging technologies,Q2667876
1,Machine guidance,Q6723715
2,Gustatory technology,Q17054731
3,Differential technological development,Q5275369
4,Robotic paradigm,Q7353423
...,...,...
54685,Electronic data interchange,Q690189
54687,Financial technology,NaN
54688,Genetics,Q7162
54689,Microsatellite,Q265193


In [17]:
df_et100_label

,Technology,HC_4,HC_10
0,Data integration,Digital Business,Digital Transformation
1,Cloud computing,Digital Business,Digital Transformation
2,Marketing automation,Digital Business,Digital Transformation
3,Legal technology,Digital Business,Digital Transformation
4,Financial technology,Digital Business,Digital Transformation
...,...,...,...
95,Biotechnology,Healthy Living,Biomedicine and Biomaterials
96,Nanotechnology,Healthy Living,Biomedicine and Biomaterials
97,Biomaterial,Healthy Living,Biomedicine and Biomaterials
98,Synthetic biology,Healthy Living,Biomedicine and Biomaterials


In [19]:
df_wk_id_unique_copy_test = df_wk_id_unique_copy[df_wk_id_unique_copy['Technology'].isin(df_et100_label['Technology'])]
df_wk_id_unique_copy_test[df_wk_id_unique_copy_test['Wikidata_ID'].isna()]

,Technology,Wikidata_ID
6424,Autonomous car,NaN
54687,Financial technology,NaN


In [20]:
import requests

def get_wikidata_id(wikipedia_article_name, lang='en'):
    """Fetch the Wikidata ID for a given Wikipedia article name."""
    url = f'https://{lang}.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'format': 'json',
        'titles': wikipedia_article_name,
        'prop': 'pageprops',
    }
    response = requests.get(url, params=params).json()
    page = next(iter(response['query']['pages'].values()))
    return page.get('pageprops', {}).get('wikibase_item')

def get_wikidata_info(wikidata_id):
    """Fetch Wikidata information using the Wikidata ID."""
    url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbgetentities',
        'ids': wikidata_id,
        'format': 'json',
        'languages': 'en',  # Adjust if you need information in another language
        'props': 'labels|descriptions|claims',  # Adjust based on the needed information
    }
    response = requests.get(url, params=params).json()
    return response['entities'][wikidata_id]



In [33]:
wikidata_id = get_wikidata_id('Explainable artificial intelligence')

In [34]:
wikidata_id

'Q40890078'

In [48]:
import wikipedia

wikipedia.search("Autonomous car")

['Self-driving car',
 'History of self-driving cars',
 'Waymo',
 'Apple car project',
 'Car',
 'Cruise (autonomous vehicle)',
 'Robotaxi',
 'Vehicular automation',
 'Self-driving car liability',
 'Adaptive cruise control']

In [30]:
for i in df_wk_id_unique_copy[df_wk_id_unique_copy['Wikidata_ID'].isna()]['Technology'].tolist():
    print (i)

Weak AI
Open Letter on Artificial Intelligence
Subject matter expert Turing test
Hybrid search engine
Explainable Artificial Intelligence
Spoken dialog systems
Transreality gaming
General purpose technology
Center for Responsible Nanotechnology
Biomorphic robotics
Postbiological evolution
Natural language understanding
ECOWAS Policy on Science and Technology (ECOPOST)
7 nanometer
Ian Sommerville (academic)
Hoytether
MPSoC
Redwood Robotics
History of computing/Permission
ABI Research
Archailect
Open-source software movement
Artificial neural network
Open source appropriate technology
Open Source Robotics Foundation
Two-dimensional materials
Hierarchical classifier
Training, test, and validation sets
Bio-energy with carbon capture and storage
Micro-Opto-Electro-Mechanical Systems
Artificial intelligence, situated approach
DC-BUS
Dialog system
Design Review Based on Failure Mode
Fredkin finite nature hypothesis
AI box
Smart objects
EMC-aware programming
ESFRI
Georgia Institute of Technolo

In [ ]:
add_dict = {'Weak AI':'Q17097955',
            'Explainable Artificial Intelligence':'Q40890078',
            'General purpose technology':'Q5532384',
            '':'Q192776'}

In [10]:
df_wikidata_info

,Wikidata_ID,Description,instance of,subclass of,Commons category,topic's main category,Freebase ID,Microsoft Academic ID,OpenAlex ID
0,Q2667876,Wikimedia list article,Q13406463,NaN,Emerging technologies,NaN,NaN,NaN,NaN
1,Q6723715,NaN,NaN,NaN,NaN,NaN,/m/0grqpg,NaN,NaN
2,Q17054731,NaN,NaN,NaN,NaN,NaN,/m/010hnp5v,NaN,NaN
3,Q5275369,strategy of technology governance,NaN,NaN,NaN,NaN,/m/0bbd70,2776700394,NaN
4,Q7353423,NaN,NaN,NaN,NaN,Q8670706,/m/08z8pw,60940618,C60940618
...,...,...,...,...,...,...,...,...,...
47770,Q1068473,use of technology in education to improve lear...,"Q86478515,Q112057532",Q11016,Educational technology,Q8408260,/m/067_f8,"16443162,2781350754","C2986849125,C16443162,C2909807104,C3019723476,..."
47771,Q690189,electronic communication method,NaN,"Q1172338,Q2182842",NaN,Q8946971,/m/02nvq,41525213,C41525213
47772,Q7162,"science of genes, heredity, and variation in l...","Q336,Q28598684,Q11862829",Q420,Genetics,"Q4925926,Q33094299",/m/036_2,"54355233,2910704739",C54355233
47773,Q265193,repeating sequences of 2-13 base pairs of DNA,"Q424473,Q425319",Q2587423,Microsatellites,NaN,/m/0c1z9,61320498,C61320498


In [9]:
df_wikidata_info.shape

(47775, 9)

In [ ]:


df_et_wk_info = df_wk_id_unique_copy.merge(df_wikidata_info, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

remove_instance_list = ['human','business','organization','idiom','culture','state','ancient','style',
                        'historical','video game','public company','company','type of chemical entity','entity',
                        'research institute','institute','enterprise','written work','nonprofit organization',
                        'programming language','university','website','government agency','government','agency',
                        'school','standard','award','institution','taxon','history','file',
                        'format','film','conference','aspect in a geographic region','publisher',
                        'library','outline','laboratory','tabletop role-playing game','center','episode',
                        'phenomenon','magazine','brand','project','television','instrument','industry','fictional',
                        'faculty','language','quality','experiment','society','book',
                        'activity','paradox','consortium','law','hospital','timeline','comparison','equation',
                        'park','museum','subsidiary','ship','disease','database','association','bus','event',
                        'reaction','album','college','formula','polymer','compiler','division','station','venture',
                        'canal','strategy','occupation','lecture','rule','road','service','chair','newspaper',
                        'collection','task','policy','diagram','galaxy','data set','prefix','island','publication',
                        'function','news website','literary work','scientific journal','country','zone']
remove_instance_list = [i.lower() for i in remove_instance_list]


remove_subclass_list = ['smartphone','mobile phone','formula one car','diagram','philosophy','software testing',
                        'chemical compound','ethics','programming language','strategy','error',
                        'ieee 802','fallacy','goods','iso standard','grammar','memory','as a service',
                        'definition','heater','toy']
remove_subclass_list = [i.lower() for i in remove_subclass_list]

df_descr_tech = df_et_wk_info[(~df_et_wk_info['Description'].isna()) & (df_et_wk_info['Description'].str.contains('tech'))]
df_descr_no_tech = df_et_wk_info.iloc[~df_et_wk_info.index.isin(df_descr_tech.index.tolist())]

# Part1
df_part1 = df_descr_tech.copy()
print (df_part1.shape)

# Remove instance
all_instance_of_part1 = df_part1[~df_part1['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()
instance_of_part1_list = [y for x in all_instance_of_part1 for y in x]
df_instance_part1_counts = pd.DataFrame(pd.Series(instance_of_part1_list).value_counts()).reset_index()
df_instance_part1_counts.columns = ['Wikidata_ID','Counts']
df_instance_part1_counts = df_instance_part1_counts.merge(df_instance_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

df_instance_part1_counts = df_instance_part1_counts[~df_instance_part1_counts['instance_of_label'].isna()]
df_instance_part1_counts['instance_of_label'] = df_instance_part1_counts['instance_of_label'].apply(lambda x: str(x).lower())

for i in remove_instance_list:
    df_instance_part1_counts = df_instance_part1_counts[~df_instance_part1_counts['instance_of_label'].str.contains(i)]
    
df_part1 = df_part1.loc[(df_part1['instance of'].isna()) | (df_part1['instance of'].str.contains('|'.join(df_instance_part1_counts['Wikidata_ID'].tolist()))),:]
print (df_part1.shape)

# Remove subclass
all_subclass_of_part1 = df_part1[~df_part1['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()
subclass_of_part1_list = [y for x in all_subclass_of_part1 for y in x]
df_subclass_part1_counts = pd.DataFrame(pd.Series(subclass_of_part1_list).value_counts()).reset_index()
df_subclass_part1_counts.columns = ['Wikidata_ID','Counts']
df_subclass_part1_counts = df_subclass_part1_counts.merge(df_subclass_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

df_subclass_part1_counts = df_subclass_part1_counts[~df_subclass_part1_counts['subclass_of_label'].isna()]
df_subclass_part1_counts['subclass_of_label'] = df_subclass_part1_counts['subclass_of_label'].apply(lambda x: str(x).lower())
for i in remove_subclass_list:
    df_subclass_part1_counts = df_subclass_part1_counts[~df_subclass_part1_counts['subclass_of_label'].str.contains(i)]
    
df_part1 = df_part1.loc[(df_part1['subclass of'].isna()) | (df_part1['subclass of'].str.contains('|'.join(df_subclass_part1_counts['Wikidata_ID'].tolist()))),:]
print (df_part1.shape)

# Part2
df_part2 = df_descr_no_tech.copy()
print (df_part2.shape)

# Remove instance
all_instance_of_part2 = df_part2[~df_part2['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()
instance_of_part2_list = [y for x in all_instance_of_part2 for y in x]
df_instance_part2_counts = pd.DataFrame(pd.Series(instance_of_part2_list).value_counts()).reset_index()
df_instance_part2_counts.columns = ['Wikidata_ID','Counts']
df_instance_part2_counts = df_instance_part2_counts.merge(df_instance_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

df_instance_part2_counts = df_instance_part2_counts[~df_instance_part2_counts['instance_of_label'].isna()]
df_instance_part2_counts['instance_of_label'] = df_instance_part2_counts['instance_of_label'].apply(lambda x: str(x).lower())

for i in remove_instance_list:
    df_instance_part2_counts = df_instance_part2_counts[~df_instance_part2_counts['instance_of_label'].str.contains(i)]
    
df_part2 = df_part2.loc[(df_part2['instance of'].isna()) | (df_part2['instance of'].str.contains('|'.join(df_instance_part2_counts['Wikidata_ID'].tolist()))),:]
print (df_part2.shape)

# Remove subclass
all_subclass_of_part2 = df_part2[~df_part2['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()
subclass_of_part2_list = [y for x in all_subclass_of_part2 for y in x]
df_subclass_part2_counts = pd.DataFrame(pd.Series(subclass_of_part2_list).value_counts()).reset_index()
df_subclass_part2_counts.columns = ['Wikidata_ID','Counts']
df_subclass_part2_counts = df_subclass_part2_counts.merge(df_subclass_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

df_subclass_part2_counts = df_subclass_part2_counts[~df_subclass_part2_counts['subclass_of_label'].isna()]
df_subclass_part2_counts['subclass_of_label'] = df_subclass_part2_counts['subclass_of_label'].apply(lambda x: str(x).lower())
for i in remove_subclass_list:
    df_subclass_part2_counts = df_subclass_part2_counts[~df_subclass_part2_counts['subclass_of_label'].str.contains(i)]
    
df_part2 = df_part2.loc[(df_part2['subclass of'].isna()) | (df_part2['subclass of'].str.contains('|'.join(df_subclass_part2_counts['Wikidata_ID'].tolist()))),:]
print (df_part2.shape)

df_wikidata_filter = pd.concat([df_part1, df_part2])
print (df_wikidata_filter.shape)